In [ ]:
import os
import json
from PIL import Image

def predict_caption(image_paths):
    images = []
    for image_path in image_paths:
        i_image = Image.open(image_path)
        if i_image.mode != "RGB":
            i_image = i_image.convert(mode="RGB")
        images.append(i_image)

    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    output_ids = model.generate(pixel_values, **gen_kwargs)

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]
    return preds[0]

def generate_json_file(folder_path):
    image_data = []
    frame_id = 1

    # Recursive function to traverse folders inside folders
    def traverse_folder(current_path):
        nonlocal frame_id

        for filename in os.listdir(current_path):
            file_path = os.path.join(current_path, filename)

            if os.path.isdir(file_path):
                # Recursively traverse subfolder
                traverse_folder(file_path)
                # Reset frame ID to 1 for each new folder
                frame_id = 1
            elif filename.endswith((".jpg", ".png", ".jpeg")):
                # Predict caption for the frame
                caption = predict_caption([file_path])  # Pass the frame path as a list

                # Create image data dictionary
                image_info = {
                    "frame_path": file_path.replace("\\\\", "\\"),
                    "frame_id": frame_id,
                    "caption": caption
                }
                # Increment frame ID
                frame_id += 1

                # Add image data to the list
                image_data.append(image_info)

    # Start traversing from the root folder
    traverse_folder(folder_path)

    # Convert image data to JSON
    json_data = json.dumps(image_data, indent=4)

    # Write JSON data to a file
    with open("Final_frames_to_json_format.json", "w") as file:
        file.write(json_data)

# Provide the folder path containing image frames
folder_path = "C:\\Users\\admin\\Desktop\\SCET_Major\\frmes to json\\PreprocessDataset"  # Make sure to specify the complete folder path

# Generate JSON file
generate_json_file(folder_path)
